#Here I want share How to Make  GAU (Gating Attention Units) Layers
#for everyone who wanna learn create more light Transformers  

In [3]:
import littlelearn as ll
from littlelearn import DeepLearning as dl
import math
import torch
from torch import nn
from tensorflow import keras
import tensorflow as tf

# With LittleLearn



to be honest GAU is implemented at layers API, but here i want sharing how we can make it by manual.


In [5]:
layers = dl.layers.GAU(32,use_causalmask=True)
test = ll.rand(1,1,32)
layers(test,test,test)

Tensor(shape=(1, 1, 32), dtype=<class 'jax.numpy.float32'>, device=cpu, requires_grad=False)
[[[-0.21995004  0.11773483  0.06039233 -0.06927146  0.19805276
    0.2159711   0.11858805  0.0735042  -0.37021443  0.23604305
   -0.08045566 -0.02607459  0.23318562  0.17026182 -0.01790002
    0.1831313   0.01150634 -0.10715868 -0.04895158  0.04067403
    0.06841192 -0.10092467 -0.2370058   0.05023849 -0.03271296
    0.07916956  0.3122727  -0.05678637  0.12025668 -0.06060823
    0.02510053 -0.06312887]]]

In [6]:
import math

#Oke lets build Manual GAU

In [7]:
class LLGAU (dl.layers.Component) :
  def __init__ (self,embed_dim) :
    super().__init__()
    std = math.sqrt(2/(embed_dim * 2))
    self.wq = dl.layers.Parameter(
        tensor=ll.normal(mean=0,std=std,shape=(embed_dim,embed_dim))
    )
    self.wk = dl.layers.Parameter(
        tensor=ll.normal(mean=0,std=std,shape=(embed_dim,embed_dim))
    )
    self.wv = dl.layers.Parameter(
        tensor=ll.normal(mean=0,std=std,shape=(embed_dim,embed_dim))
    )
    self.wgate = dl.layers.Parameter(
        tensor=ll.normal(mean=0,std=std,shape=(embed_dim,embed_dim))
    )
    self.wout = dl.layers.Parameter(
        tensor=ll.normal(mean=0,std=std,shape=(embed_dim,embed_dim))
    )

  def count_score (self,q,k,v) :
    score = ll.matmul(q,k,transpose_b=True) / k.shape[-1] ** 0.5
    score = dl.activations.softplus(score)
    out = ll.matmul(score,v)
    return out

  def forwardpass(self,q,k,v) :
    b,s,d = q.shape
    Q = ll.matmul(q,self.wq)
    K = ll.matmul(k,self.wk)
    V = ll.matmul(v,self.wv)

    gating = ll.matmul(q,self.wgate)
    gating = dl.activations.sigmoid(gating)
    out = self.count_score(Q,K,V)
    out = out * gating
    out = ll.matmul(out,self.wout)

    return out

testing

In [10]:
logits = ll.randn((1,1,32),requires_grad=True)
layers = LLGAU(32)
layers.train(True)

name : LLGAU status : active || requires_grad : True


In [11]:
out = layers(logits,logits,logits)
out

Tensor(shape=(1, 1, 32), dtype=<class 'jax.numpy.float32'>, device=cpu, requires_grad=True)
[[[ 3.6267908  -1.3859645  -1.5053034   5.198196   -0.4432551
    4.5121636   5.9736137  -6.0510736   1.4292274  -1.6760142
   -0.5340683  -0.08145965 -6.4350147   7.6194444  -2.1674361
   -0.03600753 -5.4629164  -4.6233478  -1.9080942  -2.636688
   -1.2105691   1.2777492  -8.243667    6.5960474   5.071932
   -3.317241    5.11329    -5.6657777   1.2207596   3.855531
   -1.169121    0.07661773]]]

In [12]:
out.backwardpass()

In [17]:
out.reset_grad()

# With Torch

In [18]:
class TorchGAU (nn.Module) :
  def __init__(self,embed_dim) :
    super().__init__()
    std = math.sqrt(2/(embed_dim * 2))
    self.wq = nn.Parameter(torch.normal(mean=0,std=std,size=(embed_dim,embed_dim)))
    self.wk = nn.Parameter(torch.normal(mean=0,std=std,size=(embed_dim,embed_dim)))
    self.wv = nn.Parameter(torch.normal(mean=0,std=std,size=(embed_dim,embed_dim)))
    self.wgate = nn.Parameter(torch.normal(mean=0,std=std,size=(embed_dim,embed_dim)))
    self.wout = nn.Parameter(torch.normal(mean=0,std=std,size=(embed_dim,embed_dim)))

  def count_attention (self,q,k,v) :
    score = torch.matmul(q,k.transpose(-2,-1)) / k.shape[-1] ** 0.5
    score = torch.nn.functional.softplus(score)
    out = torch.matmul(score,v)
    return out

  def forward(self,q,k,v) :
    b,s,d = q.shape
    Q = torch.matmul(q,self.wq)
    K = torch.matmul(k,self.wk)
    V = torch.matmul(v,self.wv)

    gating = torch.matmul(q,self.wgate)
    gating = torch.sigmoid(gating)
    out = self.count_attention(Q,K,V)
    out = out * gating
    out = torch.matmul(out,self.wout)
    return out

Testing

In [19]:
layers = TorchGAU(32)
layers.train(True)

TorchGAU()

In [20]:
logits = torch.rand(1,1,32)

In [21]:
out = layers(logits,logits,logits)
out

tensor([[[ 0.0010,  0.0017, -0.0089,  0.0996,  0.2169,  0.2483, -0.2100,
          -0.2156, -0.1056, -0.1064, -0.1626, -0.2534, -0.3525, -0.0466,
           0.0683,  0.2508,  0.1061, -0.2134, -0.0947, -0.6474, -0.2895,
          -0.2039,  0.3118,  0.0417, -0.0400,  0.0580, -0.0066,  0.0071,
           0.0354, -0.2313,  0.2615,  0.1075]]], grad_fn=<UnsafeViewBackward0>)

With Tensorflow

In [23]:
@keras.utils.register_keras_serializable()
class TfGAU (keras.layers.Layer) :
  def __init__(self,embed_dim) :
    super().__init__()
    std = math.sqrt(2/(embed_dim * 2))
    self.wq = keras.initializers.RandomNormal(mean=0,stddev=std)(shape=(embed_dim,embed_dim))
    self.wk = keras.initializers.RandomNormal(mean=0,stddev=std)(shape=(embed_dim,embed_dim))
    self.wv = keras.initializers.RandomNormal(mean=0,stddev=std)(shape=(embed_dim,embed_dim))
    self.wgate = keras.initializers.RandomNormal(mean=0,stddev=std)(shape=(embed_dim,embed_dim))
    self.wout = keras.initializers.RandomNormal(mean=0,stddev=std)(shape=(embed_dim,embed_dim))

  def count_attention (self,q,k,v) :
    score = tf.matmul(q,k,transpose_b=True) / k.shape[-1] ** 0.5
    score = tf.nn.softplus(score)
    out = tf.matmul(score,v)
    return out

  def call (self,q,k,v) :
    b,s,d = q.shape
    Q = tf.matmul(q,self.wq)
    K = tf.matmul(k,self.wk)
    V = tf.matmul(v,self.wv)

    gating = tf.matmul(q,self.wgate)
    gating = tf.nn.sigmoid(gating)
    attn = self.count_attention(Q,K,V)
    out = attn * gating
    out = tf.matmul(out,self.wout)
    return out

Testing

In [24]:
layers = TfGAU(32)
logits = tf.random.normal((1,1,32))

In [25]:
layers(logits,logits,logits)

<tf.Tensor: shape=(1, 1, 32), dtype=float32, numpy=
array([[[-0.18055338, -0.12918295,  0.9146644 , -0.19612189,
         -0.49990502, -0.2993888 , -0.7781618 , -0.14139004,
         -0.6453798 ,  0.05248633,  0.5470862 ,  0.13061911,
          0.01359482,  0.03959207,  0.16944441, -0.35442755,
         -0.24433155,  0.62876767, -0.4541697 ,  0.33135998,
          0.8075741 , -0.3688897 ,  0.25799742, -0.08735079,
         -0.72536975, -0.50624907,  0.05734975,  0.44088173,
         -0.66767085, -0.3967695 , -0.01072117,  0.18601109]]],
      dtype=float32)>

oke, lest random choice for training testing with Transformers Model

In [26]:
import random
print({
    "1" : "LittleLearn",
    "2" : "Torch",
    "3" : "Tensorflow"

})

print(f"we will training with {random.choice(['1','2','3'])}")

{'1': 'LittleLearn', '2': 'Torch', '3': 'Tensorflow'}
we will training with 2


oke, lest training with Torch. i will edit torchGAU for make it suppport DecoderOnly Transformers

In [36]:
class TorchGAU (nn.Module) :
  def __init__(self,embed_dim) :
    super().__init__()
    std = math.sqrt(2/(embed_dim * 2))
    self.wq = nn.Parameter(torch.normal(mean=0,std=std,size=(embed_dim,embed_dim)))
    self.wk = nn.Parameter(torch.normal(mean=0,std=std,size=(embed_dim,embed_dim)))
    self.wv = nn.Parameter(torch.normal(mean=0,std=std,size=(embed_dim,embed_dim)))
    self.wgate = nn.Parameter(torch.normal(mean=0,std=std,size=(embed_dim,embed_dim)))
    self.wout = nn.Parameter(torch.normal(mean=0,std=std,size=(embed_dim,embed_dim)))

  def count_attention (self,q,k,v) :
    score = torch.matmul(q,k.transpose(-2,-1)) / k.shape[-1] ** 0.5
    mask = 1 - torch.tril(torch.ones(score.shape),diagonal=0)
    mask = mask * -1e9
    score = score + mask
    score = torch.nn.functional.softplus(score)
    out = torch.matmul(score,v)
    return out

  def forward(self,q,k,v) :
    b,s,d = q.shape
    Q = torch.matmul(q,self.wq)
    K = torch.matmul(k,self.wk)
    V = torch.matmul(v,self.wv)

    gating = torch.matmul(q,self.wgate)
    gating = torch.sigmoid(gating)
    out = self.count_attention(Q,K,V)
    out = out * gating
    out = torch.matmul(out,self.wout)
    return out

In [27]:
data = ["Happy new years everyone, this is a tutorial from me spesial for last night in  2025  . i hope i will get job, and anyone learn for this tutorial can get job to. i hope anyone here can find with who ones precious.. "]

In [32]:
tokenizer = ll.preprocessing.Tokenizer()
tokenizer.fit_on_texts(data)
tokenizer.fit_on_texts(["sos" , "eos"])
x_train = tokenizer.texts_to_sequences(["sos " + data[0]])
y_train = tokenizer.texts_to_sequences([data[0] + " eos"])


In [33]:
x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)

In [50]:
class TransformersBlock (nn.Module) :
  def __init__(self,d_model,droprate) :
    super().__init__()
    self.norm1 = nn.RMSNorm(d_model)
    self.norm2 = nn.RMSNorm(d_model)
    self.attention = TorchGAU(d_model)
    self.ffn = nn.Sequential(
        nn.Linear(d_model,4*d_model),
        nn.GELU(),
        nn.Linear(4*d_model,d_model)
    )
    self.dropout1 = nn.Dropout(droprate)
    self.dropout2 = nn.Dropout(droprate)

  def forward(self,x) :
    norm1 = self.norm1(x)
    attn = self.attention(norm1,norm1,norm1)
    attn = self.dropout1(attn)
    x = x + attn

    norm2 = self.norm2(x)
    ffn = self.ffn(norm2)
    ffn = self.dropout2(ffn)
    x = x + ffn
    return x

class DecoderOnlyTransformers (nn.Module) :
  def __init__(self,vocab_size,d_model,n_layers,droprate,maxpos) :
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,d_model)
    self.layers = nn.ModuleList([
        TransformersBlock(d_model,droprate) for _ in range(n_layers)
    ])
    self.pos_learn = nn.Embedding(maxpos,d_model )
    self.linear = nn.Linear(d_model,vocab_size,bias=False)
    self.scale = math.sqrt(float(d_model))

  def forward(self,x) :
    b,s = x.shape
    x = self.embedding(x)
    x = x * self.scale
    pos = torch.arange(s)
    pos  = self.pos_learn(pos)
    pos = pos.unsqueeze(0)

    x = x + pos
    for block in self.layers :
      x = block(x)

    x = self.linear(x)
    return x

In [91]:
model = DecoderOnlyTransformers(50,32,3,0.1,50)
model.train(True)

DecoderOnlyTransformers(
  (embedding): Embedding(50, 32)
  (layers): ModuleList(
    (0-2): 3 x TransformersBlock(
      (norm1): RMSNorm((32,), eps=None, elementwise_affine=True)
      (norm2): RMSNorm((32,), eps=None, elementwise_affine=True)
      (attention): TorchGAU()
      (ffn): Sequential(
        (0): Linear(in_features=32, out_features=128, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=128, out_features=32, bias=True)
      )
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
  )
  (pos_learn): Embedding(50, 32)
  (linear): Linear(in_features=32, out_features=50, bias=False)
)

In [92]:
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters())

In [94]:
for i in range(100) :
  y_pred = model(x_train)
  loss = loss_fn(y_pred.transpose(-2,-1),y_train)
  optim.zero_grad()
  loss.backward()
  optim.step()

  print(loss.item())

0.024616535753011703
0.09218788146972656
0.028056371957063675
0.03951152041554451
0.04926704242825508
0.017196714878082275
0.015601158142089844
0.03552627190947533
0.012745165266096592
0.018112873658537865
0.023993780836462975
0.05996466800570488
0.04168415442109108
0.06342701613903046
0.02466748282313347
0.05139461159706116
0.018816890195012093
0.05477495491504669
0.042482130229473114
0.029024329036474228
0.022892311215400696
0.014082225039601326
0.017684251070022583
0.008813712745904922
0.016623396426439285
0.03588951751589775
0.01944906823337078
0.01593070477247238
0.019225722178816795
0.029220733791589737
0.011034054681658745
0.014312622137367725
0.010603860020637512
0.012488635256886482
0.011321105062961578
0.011183390393853188
0.012063626199960709
0.01202724315226078
0.014223952777683735
0.00986446812748909
0.01321214996278286
0.014996462501585484
0.015858441591262817
0.015854496508836746
0.011062695644795895
0.019584642723202705
0.010497672483325005
0.013602307066321373
0.007243

In [95]:
model.eval()

DecoderOnlyTransformers(
  (embedding): Embedding(50, 32)
  (layers): ModuleList(
    (0-2): 3 x TransformersBlock(
      (norm1): RMSNorm((32,), eps=None, elementwise_affine=True)
      (norm2): RMSNorm((32,), eps=None, elementwise_affine=True)
      (attention): TorchGAU()
      (ffn): Sequential(
        (0): Linear(in_features=32, out_features=128, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=128, out_features=32, bias=True)
      )
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
  )
  (pos_learn): Embedding(50, 32)
  (linear): Linear(in_features=32, out_features=50, bias=False)
)

In [96]:
def decod(texts):
    r = texts
    seq = tokenizer.texts_to_sequences([texts])[0]


    input_ = torch.tensor([seq], dtype=torch.long)


    start = len(seq)

    for _ in range(start, 43):

        with torch.no_grad():
            logits = model(input_)


        logits = logits[:, -1, :]
        logits = torch.softmax(logits, dim=-1)

        token = torch.multinomial(logits, num_samples=1)
        token_id = token.item()


        r += " " + tokenizer.index_word.get(token_id, "")


        input_ = torch.cat([input_, token], dim=1)
    return r


In [101]:
decod("Happy new years")

'Happy new years everyone this is a tutorial from me spesial for last night in 2025 i hope i will get job and anyone here can find anyone here anyone here job to i hope anyone here can find with who ones precious'

In [86]:
tokenizer.word_index

{'happy': 1,
 'new': 2,
 'years': 3,
 'everyone': 4,
 'this': 5,
 'is': 6,
 'a': 7,
 'tutorial': 8,
 'from': 9,
 'me': 10,
 'spesial': 11,
 'for': 12,
 'last': 13,
 'night': 14,
 'in': 15,
 '2025': 16,
 'i': 17,
 'hope': 18,
 'will': 19,
 'get': 20,
 'job': 21,
 'and': 22,
 'anyone': 23,
 'learn': 24,
 'can': 25,
 'to': 26,
 'here': 27,
 'find': 28,
 'with': 29,
 'who': 30,
 'ones': 31,
 'precious': 32,
 'sos': 33,
 'eos': 34}